# Tugas 5 : LSA Topic Modelling

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Import data

In [ ]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/wahyuarilsaputra/dataset/main/Data_KomentarYoutube.csv')
df

,komentar,hasil komentar
0,"ID\nSkip navigation\nSign in\nPilpres 2024, PD...",ini pilpres pdi p megawati umumkan capres ...
1,Kami hanya mendukung partai politik yang konsi...,kami hanya mendukung partai politik yang konsi...
2,Dari ucapan yang disampaikan Pak GP dan PDIP k...,dari ucapan yang disampaikan pak gp dan pdip k...
3,Kalau saya lebih baik mendukung pak mahpud MD ...,kalau saya lebih baik mendukung pak mahpud md ...
4,Mantap inilah yg di tunggu tunggu Olh rakyat ...,mantap inilah yang di tunggu tunggu olh rakyat...
...,...,...
795,Presiden Fokus pada Sisa Jabatan..jangan offsi...,presen fokus pada sisa jabatan jangan offse ur...
796,"Biasa aja , gak heboh atas pengumuman ini .",biasa saja enggak heboh atas pengumuman ini
797,MEMANG HARUS KITA HARUS PILIH CAPRES YG MAU ME...,memang harus kita harus pilih capres yang mau ...
798,hati hati merah,hati hati merah


## Instal NLTK

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)

True

## Tokenisasi Data

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['hasil komentar'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

## Menentukan bobot per topik

In [ ]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topik "+str(index)+": ",top_terms_key)

Topik 0:  [('ganjar', 0.4760470618894732), ('partai', 0.32776174182260515)]
Topik 1:  [('ganjar', 0.7105860367049839), ('pranowo', 0.15529183789385576)]
Topik 2:  [('pdip', 0.506965030091819), ('golkar', 0.14204125637777623)]
Topik 3:  [('pdi', 0.6110632580391213), ('p', 0.5796936998199563)]


In [ ]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["hasil komentar"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,hasil komentar,Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,ini pilpres pdi p megawati umumkan capres ...,0.183385,-0.040972,-0.021575,0.000449,0
1,kami hanya mendukung partai politik yang konsi...,0.113672,-0.120966,-0.046930,-0.027069,0
2,dari ucapan yang disampaikan pak gp dan pdip k...,0.178766,-0.133017,0.047605,-0.088116,0
3,kalau saya lebih baik mendukung pak mahpud md ...,0.162526,-0.040454,-0.386367,-0.101077,0
4,mantap inilah yang di tunggu tunggu olh rakyat...,0.145753,-0.028339,-0.130629,0.017401,0
...,...,...,...,...,...,...
795,presen fokus pada sisa jabatan jangan offse ur...,0.201750,-0.145467,0.051823,-0.050311,0
796,biasa saja enggak heboh atas pengumuman ini,0.003646,-0.003895,-0.000819,-0.001875,0
797,memang harus kita harus pilih capres yang mau ...,0.117455,-0.045284,-0.072395,-0.026255,0
798,hati hati merah,0.020583,-0.000998,-0.036016,-0.001319,0


In [ ]:
df_lsa['Topik'].value_counts()

0    653
3     58
2     45
1     44
Name: Topik, dtype: int64